### 모델 정보

In [ ]:
# !pip install xgboost
# !pip install lightgbm
!pip install catboost

In [ ]:
# 선형 모델
# 로지스틱 -> 데이터가 선형 관계를 가질 때
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# 릿지 -> L2, 데이터의 차원이 많고 다중공선성이 존재할 때
from sklearn.linear_model import Ridge
model = Ridge(alpha=1.0)
# 라쏘 -> L1, 불필요한 특성 자동 제거
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1)
# 엘라스틱넷 -> LogisticRegression + Lasso
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=0.1, l1_ratio=0.5)

# 비선형 모델
# 랜덤포레스트 -> 여러 개의 결정 트리를 조합, 이상치에 강함
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
model = RandomForestRegressor(n_estimator=100, random_state=42)
# 그래디언트 부스팅 -> 데이터가 복잡하고 비선형적.
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
# SVM -> 비선형 분류 모델
from sklearn.svm import SVC
model = SVC(kernel='rbf', C=1.0)

# 대용량 데이터에서 강력한 성능을 보이는 모델
# xgboost
from xgboost import XGBRegressor
from xgboost import XGBClassifier
model = XGBRegressor(n_estimators= 100, learning_rate=0.1)
# lightgbm -> 
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_estimators=100, learning_rate=0.1)
# catboost -> 범주형 변수 처리에 강함
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
model = CatBoostRegressor(n_estimators=100, learning_rate=0.1, verbose=0)

# 여러 모델을 사용
- 분류 모델

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv')

# 모델 학습에 필요없는 특정 변수 제거하기
df_processing = df.drop('name', axis=1,inplace=False)

# 데이터 전처리
# 정규화 MinMax
df_norm = preprocessing.minmax_scale(df_processing)
df_processed = pd.DataFrame(df_norm)
df_processed.columns = df_processing.columns

# 종속변수와 독립변수 분리
feature_columns = df_processed.columns.difference(["stat"]) # 종속 변수
X = df_processed[feature_columns]
y = df_processed['status'].astype('category')

# 테스트와 트레인셋 분리
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.2, stratify=y, random_state=42)

# 테스트할 모델 리스트
models = {
    "LogisticRegression" : LogisticRegression(),
    "Random Forest" : RandomForestClassifier(n_estimators=100),
    "Gradient Boosting" : GradientBoostingClassifier(n_estimators=100),
    "Support Vector Machine" : SVC(kernel='rbf'),
    "XGBoost" : XGBClassifier(n_estimators=100, user_label_encoder=False,eval_metric='logloss' ,learning_rate=0.1)
    
}

# 결과를 저장할 데이터프레임
result = []

# 모델 학습 및 평가
for name, model in models.items() :
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    result.append({'Model':name, 'Accuracy':acc})

# 결과 출력
df_result = pd.DataFrame(result).sort_values(by='Accuracy', acending=False)
print(df_result)

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv')

# 모델 학습에 필요없는 특정 변수 제거하기
df_processing = df.drop('name', axis=1,inplace=False)

# 결측치
df['Age'].fillna(df['Age'].median(), inplace=True)  # 나이 결측치 → 중앙값으로 대체
df['Income'].fillna(df['Income'].mean(), inplace=True)  # 소득 결측치 → 평균값으로 대체
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)  # 성별 결측치 → 최빈값으로 대체

# IQR 계산
Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1

# 이상치 기준 설정
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치 필터링
df_no_outliers = df[(df['Age'] >= lower_bound) & (df['Age'] <= upper_bound)]
print(df_no_outliers)

# 'sex' 변수를 원-핫 인코딩 적용
df_processing = pd.get_dummies(df_processing, columns=['sex'], drop_first=True)  # drop_first=True로 다중공선성 방지

# 데이터 전처리
# 정규화 MinMax
df_norm = preprocessing.minmax_scale(df_processing)
df_processed = pd.DataFrame(df_norm)
df_processed.columns = df_processing.columns

# 종속변수와 독립변수 분리
feature_columns = df_processed.columns.difference(["stat"]) # 종속 변수
X = df_processed[feature_columns]
y = df_processed['status'].astype('category')

# 테스트와 트레인셋 분리
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.2, stratify=y, random_state=42)

# 테스트할 모델 리스트
models = {
    "LogisticRegression" : LogisticRegression(),
    "Random Forest" : RandomForestClassifier(n_estimators=100),
    "Gradient Boosting" : GradientBoostingClassifier(n_estimators=100),
    "Support Vector Machine" : SVC(kernel='rbf'),
    "XGBoost" : XGBClassifier(n_estimators=100, user_label_encoder=False,eval_metric='logloss' ,learning_rate=0.1)
}

# 결과를 저장할 데이터프레임
result = []
model_predictions = {}
# 모델 학습 및 평가
for name, model in models.items() :
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)
    acc = accuracy_score(test_y, pred_y)
    result.append({'Model':name, 'Accuracy':acc})

    model_predictions[name] = pred_y

# 결과 출력
df_result = pd.DataFrame(result).sort_values(by='Accuracy', ascending=False)
print(df_result)
best_model = df_result.iloc[0]['Model']
print(best_model)
pd.DataFrame(model_predictions[best_model]).to_csv('result.csv', index=False, encoding='utf-8')




# 회귀 문제
- 모델 이름 변경
- 평가지표 변경 -> mean_squared_error(), mean_absolute_error(), r2_score()
- 종속변수 타입 -> 'float'

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor

# 📌 1. 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv')

# 📌 2. 필요없는 컬럼 제거
df_processing = df.drop(columns=['name'])

# 📌 3. 'sex' 변수가 있다면 원-핫 인코딩 적용
if 'sex' in df_processing.columns:
    df_processing = pd.get_dummies(df_processing, columns=['sex'], drop_first=True)

# 📌 4. 데이터 정규화 (MinMax 정규화)
df_norm = preprocessing.minmax_scale(df_processing)
df_processed = pd.DataFrame(df_norm, columns=df_processing.columns)

# 📌 5. 종속변수와 독립변수 분리 (회귀 문제이므로 `y`를 float으로 변환)
X = df_processed.drop(columns=["status"])  # 독립 변수
y = df_processed["status"].astype("float")  # 회귀 문제에서는 숫자형으로 유지

# 📌 6. 학습/테스트 데이터 분리
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 7. 회귀 모델 리스트
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100),
    "Support Vector Machine": SVR(),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1)
}

# 📌 8. 결과를 저장할 데이터프레임
results = []
model_predictions = {}

# 📌 9. 모델 학습 및 평가
for name, model in models.items():
    model.fit(train_x, train_y)
    pred_y = model.predict(test_x)

    mse = mean_squared_error(test_y, pred_y)  # 평균제곱오차 (MSE)
    mae = mean_absolute_error(test_y, pred_y)  # 평균절대오차 (MAE)
    r2 = r2_score(test_y, pred_y)  # 결정계수 (R²)

    results.append({'Model': name, 'MSE': mse, 'MAE': mae, 'R2 Score': r2})
    model_predictions[name] = pred_y

# 📌 10. 결과 정렬 후 가장 좋은 모델 선택 (R2 Score 기준)
df_result = pd.DataFrame(results).sort_values(by='R2 Score', ascending=False)
print(df_result)

best_model = df_result.iloc[0]['Model']
print(f"🏆 가장 높은 성능을 보인 모델: {best_model}")

# 📌 11. 원본 df에 최적 모델의 예측값 추가
df.loc[test_x.index, f"Prediction_{best_model}"] = model_predictions[best_model]

# 📌 12. 최적 모델 저장
joblib.dump(models[best_model], "best_regression_model.pkl")
print(f"✅ 최적 회귀 모델 ({best_model})이 'best_regression_model.pkl'로 저장되었습니다!")

# 📌 13. 결과 CSV 저장
df.to_csv('regression_result.csv', index=False, encoding='utf-8')
print("✅ 예측 결과가 'regression_result.csv' 파일로 저장되었습니다!")